In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("mySQLite") \
    .config("spark.jars", "/path/to/sqlite-jdbc-3.34.0.jar") \
    .getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
# Define URL JDBC và các thuộc tính kết nối
jdbc_url = "jdbc:sqlite:/myProject/yelp.db"  # URL JDBC cho cơ sở dữ liệu SQLite
connection_properties = {
    "driver": "org.sqlite.JDBC"  # Chỉ định lớp driver cho SQLite
}

df = spark.read.jdbc(url=jdbc_url, table="yelp", properties=connection_properties)

In [ ]:
df.show()

In [ ]:
df.count()

Create operation

In [ ]:
from pyspark.sql import Row

# Tạo một hàng dữ liệu mới
new_row = Row(Phone = '1284680203', Organization = 'ChickenGarden', Rating = 3, NumberReview = 100, Category = 'Chicken', State = 'AL', City = 'Oxford', Street = 'Beclin', Building = '131' )
# Tạo DataFrame mới (new_df) chỉ có new_row
new_df = spark.createDataFrame([new_row], schema=df.schema)
# Thêm DataFrame mới vào DataFrame hiện tại (df):
df = df.union(new_df)

df.tail(3)
df.count()

Delete Operation

In [ ]:
df = df.filter(~((df.Phone == '13342221398') & (df.Organization == 'Happy Kitchen') & (df.Rating == 3.5) & (df.NumberReview == 11) & (df.Category == 'Delivery') & (df.State == 'AL') & (df.City == 'Andalusia') & (df.Street == ' 700 Western Gate') & (df.Building == '700')))

# Show the filtered DataFrame
df.show()
print(df.count())

#Row 12

Update operation

In [ ]:
from pyspark.sql.functions import when

# Update the DataFrame to change the NumberReview columns
df = df.withColumn('NumberReview', when((df.Organization == 'Pizza Hut') & (df.City == 'Alexander City') & (df.Rating == 3), 7).otherwise(df.NumberReview))

# Show the updated DataFrame
df.head(10)


Read operation

In [ ]:
filtered_df = df.filter(df.Rating > 4)

# Show the filtered DataFrame
filtered_df.show()
print(filtered_df.count())

The performance of some queries

In [ ]:
%%time
df.createOrReplaceTempView("yelp")
spark.sql("SELECT * FROM yelp").show(5)

In [ ]:
%%time
spark.sql("SELECT * FROM yelp WHERE Rating > 4").show()

In [ ]:
%%time
spark.sql("SELECT City, SUM(NumberReview) as TotalNumberReview FROM yelp WHERE Rating > 4 GROUP BY City").show()

# Hiển thị tổng số lượt đánh giá (TotalNumberReview) cho mỗi City có Rating lớn hơn 4.

In [ ]:
%%time
spark.sql("SELECT City, SUM(NumberReview) as TotalNumberReview FROM yelp WHERE Rating > 4 GROUP BY City HAVING TotalNumberReview > 100").show()

# Thực hiện câu lệnh SQL để tính tổng giá trị (SUM(NumberReview)) của các hàng có Rating lớn hơn 4
# Nhóm kết quả theo City, và chỉ hiển thị các nhóm có SUM(NumberReview) lớn hơn 100.